<a href="https://colab.research.google.com/github/GBakalkinOAI/DDLS2024/blob/main/CellxGene_Census_scVI_Monocytes_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet scvi-colab
!pip install --quiet cellxgene-census
!pip install --quiet pybiomart
from scvi_colab import install

install()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.0/129.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.5 MB/s eta 0:00:00
ERROR: pip's depe

In [ ]:
# Import necessary libraries
import cellxgene_census
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scvi

In [ ]:
with cellxgene_census.open_soma(census_version="2024-07-01") as census:

    cell_metadata = cellxgene_census.get_obs(
        census,
        "homo_sapiens",
        # value_filter = "sex == 'male' and is_primary_data == True",
        value_filter = "sex == 'male'",
        column_names = [
            "dataset_id",
            "assay", "assay_ontology_term_id", "suspension_type", # assay, cell or nucleus
            "cell_type", "cell_type_ontology_term_id",
            "development_stage", # sometimes we have age here, too many "unknown"
            "self_reported_ethnicity", # too many "unknown"
            "tissue", "tissue_general", "tissue_type",
            "disease", "disease_ontology_term_id",
            "donor_id"
            ]
    )
print(cell_metadata.columns)

Index(['dataset_id', 'assay', 'assay_ontology_term_id', 'suspension_type',
       'cell_type', 'cell_type_ontology_term_id', 'development_stage',
       'tissue_ontology_term_id', 'tissue_general',
       'tissue_general_ontology_term_id', 'disease_ontology_term_id',
       'donor_id', 'sex'],
      dtype='object')


In [ ]:
print(cell_metadata["dataset_id"].value_counts(normalize=True).head(3))
print("")
print(cell_metadata.value_counts(subset=["assay", "assay_ontology_term_id", "suspension_type"], normalize=True).head(3))
print("")
print(cell_metadata.value_counts(subset=["cell_type", "cell_type_ontology_term_id"], normalize=True).head(5))
print("")
print(cell_metadata.value_counts(subset=["development_stage"]).head(5))
print("")
#print(cell_metadata["self_reported_ethnicity"].value_counts().head(3))
#print("")
print(cell_metadata.value_counts(subset=["tissue", "tissue_ontology_term_id", "tissue_general", "tissue_general_ontology_term_id", "tissue_type"]).head(3))
print("")
print(cell_metadata.value_counts(subset=["disease", "disease_ontology_term_id"]).head(3))
print("")
print(cell_metadata.value_counts(subset=["donor_id"]).head(5))

dataset_id
f7c1c579-2dc0-47e2-ba19-8165c5a0e353    0.063174
8e10f1c4-8e98-41e5-b65f-8cd89a887122    0.060901
d6505c89-c43d-4c28-8c4f-7351a5fd5528    0.027557
Name: proportion, dtype: float64

assay      assay_ontology_term_id  suspension_type
10x 3' v3  EFO:0009922             nucleus            0.495522
10x 3' v2  EFO:0009899             cell               0.121607
10x 3' v3  EFO:0009922             cell               0.081707
Name: proportion, dtype: float64

cell_type                                                  cell_type_ontology_term_id
neuron                                                     CL:0000540                    0.190357
oligodendrocyte                                            CL:0000128                    0.055749
L2/3-6 intratelencephalic projecting glutamatergic neuron  CL:4023040                    0.054336
glutamatergic neuron                                       CL:0000679                    0.030228
astrocyte                                               

KeyError: 'self_reported_ethnicity'

In [ ]:
import cellxgene_census

with cellxgene_census.open_soma(census_version="2024-07-01") as census:

    cell_metadata = cellxgene_census.get_obs(
        census,
        "homo_sapiens",
        value_filter = "sex == 'male' and cell_type in ['microglial cell', 'neuron']",
        column_names = ["assay", "cell_type", "tissue", "tissue_general", "suspension_type", "disease"]
    )
    print(cell_metadata)

             assay        cell_type                          tissue  \
0        10x 5' v1           neuron                            lung   
1        10x 5' v1           neuron                            lung   
2        10x 5' v1           neuron                            lung   
3        10x 5' v1           neuron                            lung   
4        10x 5' v1           neuron                            lung   
...            ...              ...                             ...   
7830003  10x 3' v3  microglial cell  dorsolateral prefrontal cortex   
7830004  10x 3' v3  microglial cell  dorsolateral prefrontal cortex   
7830005  10x 3' v3  microglial cell  dorsolateral prefrontal cortex   
7830006  10x 3' v3  microglial cell  dorsolateral prefrontal cortex   
7830007  10x 3' v3  microglial cell  dorsolateral prefrontal cortex   

        tissue_general suspension_type   disease   sex  
0                 lung            cell    normal  male  
1                 lung           

In [ ]:
import cellxgene_census

census = cellxgene_census.open_soma(census_version="2024-07-01")
adata = cellxgene_census.get_anndata(
    census,
    organism = "homo_sapiens",
    measurement_name = "RNA",
    obs_value_filter = "tissue_general == 'central nervous system'",
    obs_embeddings = ["scvi"]
)

ModuleNotFoundError: No module named 'tiledb.vector_search'

In [ ]:
# Import necessary libraries
import cellxgene_census
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import scvi

In [ ]:
# Define parameters to access the dataset
emb_names = ["scvi"]  # specify the embedding you are interested in
dataset_version = "2024-07-01"  # the version of the dataset you want to use
organism = "homo_sapiens"
cell_type_query = "Monocyte"  # The cell type we want to focus on

In [ ]:
# Open the dataset and query for Monocytes with the scVI embeddings
with cellxgene_census.open_soma(census_version=dataset_version) as census:
    adata = cellxgene_census.get_anndata(
        census,
        organism=organism,
        measurement_name="RNA",
        obs_value_filter="cell_type == '{}'".format(cell_type_query),  # Selecting Monocytes only
        obs_embeddings=emb_names
    )

ModuleNotFoundError: No module named 'tiledb.vector_search'